<a href="https://colab.research.google.com/github/Rabia-Akhtr/Time-Series-Modelling-Case-Study/blob/main/Time_Series_Modelling_Case_Study_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from itertools import product

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = [7, 5]


# **Load and Read the Data**

In [ ]:
# Load data
data = pd.read_csv('/content/drive/MyDrive/Advance Research/Assignment 1/oil_price.csv')

# Preview data
data.head()

In [ ]:
print(data.columns.tolist())

In [ ]:
data.tail()

In [ ]:
data = data.dropna()

In [ ]:
# Read data
pd.concat([data.head(), data.tail()])

In [ ]:
# Null values
data.isna().sum()

In [ ]:
# Get the shape of the dataset
print(data.shape)

In [ ]:
print(data.describe())

In [ ]:
# Convert date column
data['Date'] = pd.to_datetime(data['Date'], format='%b %d, %Y')
# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

# Convert 'Price' column to numeric  and drop missing rows
data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
data.dropna(inplace=True)

# Preview the cleaned data
print(data.head())

This is the ideal structure for ARIMA modeling ✅

# **Exploratory Data Analysis (EDA)**

# **Time Series Plot**

In [ ]:
import matplotlib.pyplot as plt

def plot_oil_price(dataframe):
    """
    Plots the oil price time series using the index (Date) and the 'Price' column.
    """
    plt.figure(figsize=(7, 5), dpi=144)
    plt.plot(dataframe.index, dataframe['Price'], label='Oil Price')  # Use index, not 'Date'
    plt.title('Oil Price Time series')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.xticks(rotation=45)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()

    # Force x-axis range to match the actual data range
    plt.xlim(data.index.min(), data.index.max())

    plt.show()

plot_oil_price(data)


In [ ]:
import matplotlib.pyplot as plt

def plot_price_distribution(df, price_col='Price', bins=30):
    """

    """
    # Create a figure with two subplots (1 row, 2 columns)
    fig, (ax_hist, ax_box) = plt.subplots(
        nrows=1, ncols=2,
        figsize=(12, 5),      # width=12 inches, height=5 inches
        dpi=144               # higher DPI for crispness
    )

    # --- Histogram on the left ---
    ax_hist.hist(
        df[price_col],        # data to plot
        bins=bins,            # number of bars
        edgecolor='black'     # black border around bars
    )
    ax_hist.set_title('Histogram of Oil Prices')   # descriptive title
    ax_hist.set_xlabel('Price')                    # x-axis label
    ax_hist.set_ylabel('Frequency')                # y-axis label

    # --- Boxplot on the right ---
    ax_box.boxplot(
        df[price_col],        # data to plot
        vert=False,           # horizontal box
        patch_artist=True     # fill box with color
    )
    ax_box.set_title('Boxplot of Oil Prices')      # descriptive title
    ax_box.set_xlabel('Price')                     # x-axis label

    # Layout tweaks
    plt.tight_layout()  # adjust subplots to fit in figure area
    plt.show()          # render the figure


In [ ]:
plot_price_distribution(data)


In [ ]:

from statsmodels.tsa.seasonal import seasonal_decompose

# Rolling Mean ± Standard Deviation (30-day window)
window = 30
rol_mean = data['Price'].rolling(window=window).mean()
rol_std  = data['Price'].rolling(window=window).std()

plt.figure(figsize=(7, 5), dpi=144)
plt.plot(data['Price'],      label='Original', alpha=0.5)
plt.plot(rol_mean,           label=f'{window}-Day Rolling Mean', color='red')
plt.fill_between(rol_std.index,
                 rol_mean - rol_std,
                 rol_mean + rol_std,
                 color='red', alpha=0.2,
                 label='Rolling ±1 Std Dev')

plt.title('Original Oil Price series, Rolling Mean & Standard Deviation')
plt.xlabel('Date',fontsize=12)
plt.ylabel('Price',fontsize=12)
plt.legend()
plt.xticks(rotation=45)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.tight_layout()
plt.show()



In [ ]:
def plot_acf_pacf(price):
    """
    Plots the ACF and PACF of the given price data.

    Parameters:
    pandas Series (e.g., data['Price'])
    """
    # Plot ACF (Autocorrelation Function)
    plot_acf(data['Price'])
    plt.title('Autocorrelation (ACF) of Oil Price')
    plt.show()

    # Plot PACF (Partial Autocorrelation Function)
    plot_pacf(data['Price'])
    plt.title('Partial Autocorrelation (PACF) of Oil Price')
    plt.show()
    return

plot_acf_pacf(data['Price'])

ACF Plot Insight:
All bars are significantly above confidence bounds, even at high lags.

This means the series is non-stationary and needs differencing (d ≥ 1).

PACF Plot Insight:
Big spikes at lag 1 and 2, then smaller, random noise after.

Suggests that AR terms (p) of 1 or 2 might be appropriate.

# **ADF Test to check stationarity**

In [ ]:
# Augmented Dickey-Fuller test on the oil price data,
result = adfuller(data['Price'])

# Print the ADF statistic and p-value
print("ADF Statistic:", result[0])
print("p-value:", result[1])

The oil price data is non-stationary.

The Augmented Dickey-Fuller (ADF) test on the original oil price data produced an ADF statistic of -1.60 and a p-value of 0.484. Because the p-value is significantly greater than 0.05, we fail to reject the null hypothesis that the series has a unit root. This indicates that the series is non-stationary, consistent with the high autocorrelation observed in the ACF plot.

In practical terms, this means the statistical properties of the oil price series (such as the mean and variance) are not constant over time. Therefore, further transformation (such as differencing) is required before fitting an ARMA model.

# **Make Data Stationary**

# **Box-Cox Transformation: Find the Optimal Lambda**

Why Box-Cox?

It stabilizes the variance, which helps ARIMA fit better (especially when prices grow over time).

But it doesn’t make it stationary (ADF shows it's still not enough).

In [ ]:
from scipy.stats import boxcox

# Find the optimal Box-Cox transformation for the 'Price' column
data['boxcox_tr'], lam = boxcox(data['Price'])

# Print the optimal lambda
print('Optimal Lambda for Box-Cox transformation: %f' % lam)


In [ ]:
def plot_boxcox_transformed(series):
    """
    Plots the Box-Cox transformed time series.

    Parameters:
    pandas Series, the transformed price data
    """
    plt.figure(dpi=144)
    plt.plot(series, label='Box-Cox Transformed Price')
    plt.title('Box-Cox Transformed Oil Price Series')
    plt.xlabel('Index')
    plt.ylabel('Transformed Price')
    plt.legend()
    plt.tight_layout()
    plt.show()
    return

plot_boxcox_transformed(data['boxcox_tr'])

In [ ]:
# Take the first difference of the Box-Cox transformed oil price series
data['boxcox_tr_diff'] = data['boxcox_tr'].diff()
data.dropna(inplace=True)  # Remove first NaN
# Display the first 10 rows to show the result
print(data.head(10))


In [ ]:
# Plot the first-differenced Box-Cox transformed oil price series
plt.figure(dpi=144)
plt.plot(data['boxcox_tr_diff'], label='Box-Cox First Difference')
plt.title('Box-Cox and First Difference of Oil Price')
plt.xlabel('Index')
plt.ylabel('Differenced Transformed Price')
plt.legend()
plt.tight_layout()
plt.show()

The plot of the first-differenced Box-Cox transformed oil price series shows fluctuations around a constant mean with relatively stable variance, and no visible trend or seasonality. This visual evidence, along with the ADF test result, confirms that the series is now stationary and suitable for ARMA modeling.

In [ ]:
# re-run the ADF test
ad_fuller_result = adfuller(data['boxcox_tr_diff'][1:])

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

The Augmented Dickey-Fuller (ADF) test on the Box-Cox transformed and first-differenced oil price series yielded an ADF statistic of -5.24 and a p-value of 0.0000073. Since the p-value is much smaller than 0.05, we reject the null hypothesis of a unit root and conclude that the transformed series is stationary.

This means the series now has a constant mean and variance over time, making it suitable for ARMA modeling. This successful result confirms that the combination of Box-Cox transformation and first differencing was effective in achieving stationarity.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# Plot ACF
plt.figure(figsize=(10, 5), dpi=144)
plot_acf(data['boxcox_tr_diff'].dropna())
plt.title('ACF of Box-Cox First-Differenced Oil Price')
plt.tight_layout()
plt.show()

# Plot PACF
plt.figure(figsize=(10, 5), dpi=144)
plot_pacf(data['boxcox_tr_diff'].dropna())
plt.title('PACF of Box-Cox First-Differenced Oil Price')
plt.tight_layout()
plt.show()


The PACF plot of the Box-Cox transformed and first-differenced oil price series shows a significant spike at lag 1, with all other lags within the confidence bounds. This pattern suggests that the series can be well modeled with an autoregressive (AR) component of order 1 (i.e., p = 1) in an ARMA model.

# **Define ARMA Model and Find Best Parameters**

# **Grid search for best ARIMA(p,d,q) by AIC**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from itertools import product

# Create all (p, d, q) combinations with:
# p = 0 to 8
# d = 0, 1, 2
# q = 0 to 8
p_values = range(0, 9)
d_values = [0,1,2]
q_values = range(0, 9)
order_list = list(product(p_values, d_values, q_values))
series = data['Price'].dropna()
# Define function to optimize ARIMA
def optimize_ARIMA(series, order_list):
    results = []
    for order in order_list:
        try:
            model = ARIMA(series, order=order)
            fitted = model.fit()
            results.append({
                'order': order,
                'aic': fitted.aic
            })
            print(f"Fitted ARIMA{order} - AIC: {fitted.aic:.2f}")
        except Exception as e:
            print(f"Skipping ARIMA{order} due to error: {e}")
    return pd.DataFrame(results).sort_values(by='aic').reset_index(drop=True)

# Run optimization
result_df = optimize_ARIMA(series, order_list)

# Show best models
print("\nTop 5 ARIMA models by AIC:")
print(result_df.head())

# **Fit Best Model**

In [ ]:
# re-define the best fit model variant.
# We do this as the last 'model_fit' recorded will be for the (6,1,7) variant.

from statsmodels.tsa.arima.model import ARIMA

# Fit the best model as per AIC results
best_model = ARIMA(data['Price'].dropna(), order=(6, 1, 7))
best_model_fit = best_model.fit()


In [ ]:
#summary of the model
print(best_model_fit.summary())

The fitted ARIMA(6, 1, 7) model is:

$$
\begin{align*}
y_t' =\ &0.0672\, y_{t-1}' + 0.4336\, y_{t-2}' + 0.0067\, y_{t-3}' \\
&+ 0.5884\, y_{t-4}' - 0.0600\, y_{t-5}' - 0.7046\, y_{t-6}' \\
&- 0.0580\, \varepsilon_{t-1} - 0.5069\, \varepsilon_{t-2} - 0.0963\, \varepsilon_{t-3} \\
&- 0.5684\, \varepsilon_{t-4} + 0.0423\, \varepsilon_{t-5} + 0.6392\, \varepsilon_{t-6} \\
&+ 0.1336\, \varepsilon_{t-7} + \varepsilon_t
\end{align*}
$$


So final time series model.

# **Model Diagnostics Interpretation**

In [ ]:
best_model_fit.plot_diagnostics(figsize=(10,7))
plt.show()


In [ ]:
#get prediction start and end dates
test_data = data['Price'].dropna()

pred_start_date = test_data.index[0]
pred_end_date = test_data.index[-1]

#get the predictions and residuals
predictions = best_model_fit.predict(start=pred_start_date, end=pred_end_date)
residuals = test_data - predictions

In [ ]:
# plot the residuals

plt.figure(figsize=(10,4))
plt.plot(residuals)
plt.title('Residuals from AR Model', fontsize=20)
plt.ylabel('Error', fontsize=16)
plt.axhline(0, color='r', linestyle='--', alpha=0.2)

In [ ]:
# Plot the model prediction against the data

plt.figure(figsize=(10,4))

plt.plot(test_data)
plt.plot(predictions)

plt.legend(('Data', 'Predictions'), fontsize=16)

plt.title('ARIMA(6,1,7) model fit', fontsize=20)
plt.ylabel('Price', fontsize=16)
plt.xlabel('Month', fontsize=16)

In [ ]:
import numpy as np

# Calculate RMSE (Root Mean Squared Error) of residuals
rmse = np.sqrt(np.mean(residuals**2))

print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')


# **Forecast 24 Months Ahead with Confidence Intervals**

In [ ]:
n_forecast = 24
# Resample daily data to monthly average price
monthly_data = data['Price'].resample('M').mean()
# Drop missing values if any
monthly_data = monthly_data.dropna()
# Get forecast with confidence intervals
forecast_result = best_model_fit.get_forecast(steps=n_forecast)

# Extract forecast mean and confidence intervals
forecast_mean = forecast_result.predicted_mean
conf_int = forecast_result.conf_int()

# Create forecast index starting after last observed month
last_date = monthly_data.index[-1]
forecast_index = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=n_forecast, freq='M')

# Assign index to forecast and confidence intervals
forecast_mean.index = forecast_index
conf_int.index = forecast_index


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6), dpi=144)

# Plot historical monthly data
plt.plot(monthly_data.index, monthly_data, label='Given Monthly Data', color='blue')

# Plot forecast
plt.plot(forecast_mean.index, forecast_mean, label='24-Month Forecast', color='orange')

# Plot confidence intervals as shaded area
plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1],
                 color='gray', alpha=0.3, label='95% Confidence Interval')

plt.title('24-Month Oil Price Forecast with 95% Confidence Interval', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Oil Price (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Reconstructing conf_df
# forecast_mean: pd.Series with forecast values
# conf_int: pd.DataFrame with lower/upper confidence intervals

conf_df = pd.concat([
    forecast_mean,           # predicted values, e.g., from forecast.predicted_mean
    conf_int                 # confidence intervals, e.g., from forecast.conf_int()
], axis=1)

# Rename columns if needed
conf_df.columns = ['Predictions', 'Lower CI', 'Upper CI']


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- Historical TRAINING data (define this for 'monthly_data') ---
train_data = pd.read_csv('/content/drive/MyDrive/Advance Research/Assignment 1/oil_price.csv')
train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data.set_index('Date', inplace=True)
monthly_data = train_data['Price'].resample('ME').mean()

# --- Latest ACTUAL data ---
actual_data = pd.read_csv('/content/drive/MyDrive/Advance Research/Assignment 1/Oil price data to the current date.csv')
actual_data['Date'] = pd.to_datetime(actual_data['Date'])
actual_data.set_index('Date', inplace=True)
actuals = actual_data['Price'].resample('ME').mean()  # 'ME' fixes the warning

# --- Plot forecast vs actual data ---
plt.figure(figsize=(10, 6), dpi=144)

plt.plot(monthly_data.index, monthly_data.values, label='Given Monthly Data', color='blue')
plt.plot(conf_df.index, conf_df['Predictions'], label='24-Month Forecast', color='orange')
plt.fill_between(conf_df.index, conf_df['Lower CI'], conf_df['Upper CI'],
                 color='gray', alpha=0.3, label='95% Confidence Interval')

# Overlap only forecast/actual period
common_index = conf_df.index.intersection(actuals.index)
plt.plot(actuals.loc[common_index].index, actuals.loc[common_index].values,
         label='Current Oil Prices (to date)', color='green', linestyle='--')

plt.title('Forecast vs Current Oil Prices (Monthly)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Oil Price (USD)', fontsize=14)
plt.grid(True)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()


### Forecast vs Real Data (ARMA)

The ARMA model predicted a flat oil price trend, while real prices showed more volatility and fluctuations.  
Several actual prices fell outside the 95% confidence interval, showing that the model missed unexpected market changes.  
This highlights ARIMA's limitation in capturing non-linear,real-world dynamics despite statistical soundness.  
Better performance may require advanced models like LSTM that adapt to complex patterns.
